In [3]:
import pandas as pd

In [30]:
#LEVANTO LOS DATASETS

dataset_ff = pd.read_csv("Datasets/dataset_ff.csv")
dataset_cf = pd.read_csv("Datasets/dataset_cf.csv")
dataset_lb = pd.read_csv("Datasets/dataset_lb.csv")
dataset_am = pd.read_csv("Datasets/dataset_am.csv")


,user_id,snapshot_mes,FEC_EVENT_MAX,CANT_TOTAL_EVER,CANT_TOTAL_ULT_7D,FEC_EVENT_MENOS_7D,RATIO_VISITA_MADRUGADA,RATIO_VISITA_MANANA,RATIO_VISITA_TARDE,RATIO_VISITA_NOCHE
0,0,9,2018-09-30 20:13:42,4912,90,2018-09-23 20:13:42,0.0,0.466667,0.222222,0.311111
1,1,9,2018-09-30 20:56:50,1634,26,2018-09-23 20:56:50,0.0,0.961538,0.000000,0.038462
2,2,9,2018-09-28 00:04:58,1717,15,2018-09-21 00:04:58,0.2,0.466667,0.333333,0.000000
3,3,9,2018-09-28 16:03:37,1554,61,2018-09-21 16:03:37,0.0,0.049180,0.950820,0.000000
4,4,9,2018-09-27 22:15:33,2476,48,2018-09-20 22:15:33,0.0,0.250000,0.312500,0.437500


In [16]:
# SE BORRAN COLUMNAS DE MAS EN DATASET_FF
dataset_ff.columns

Index(['user_id', 'snapshot_mes', 'FEC_EVENT_MAX', 'CANT_TOTAL_EVER',
       'CANT_TOTAL_ULT_7D', 'FEC_EVENT_MENOS_7D', 'RATIO_VISITA_MADRUGADA',
       'RATIO_VISITA_MANANA', 'RATIO_VISITA_TARDE', 'RATIO_VISITA_NOCHE'],
      dtype='object')

In [31]:
# SE BORRAN COLUMNAS DE MAS EN DATASET_CF
dataset_cf = dataset_cf.drop(['snapshot_mes_x','snapshot_mes_y','snapshot_mes_x.1','snapshot_mes_y.1','USER_ID'], axis = 1)
dataset_cf.columns

Index(['user_id', 'qty_adopciones_1M', 'qty_adopciones_3M',
       'qty_adopciones_6M', 'qty_adopciones_9M', 'mes', 'anio',
       'qty_dias_ultima_adopcion', 'qty_dias_ultimo_page_ingreso'],
      dtype='object')

In [ ]:
# SE BORRAN COLUMNAS DE MAS EN DATASET_lb
dataset_lb = dataset_lb.drop([], axis = 1)
dataset_lb.columns

In [ ]:
# SE BORRAN COLUMNAS DE MAS EN DATASET_am
dataset_am = dataset_am.drop([], axis = 1)
dataset_am.columns

In [32]:
# MERGE DE TODOS LOS DATASETS
dataset_ff_cf = pd.merge(dataset_cf, dataset_ff, how='left', on='user_id')
dataset_ff_cf_lb = pd.merge(dataset_ff_cf, dataset_lb, how='left', on='user_id')
dataset_ff_cf_lb_am = pd.merge(dataset_ff_cf_lb, dataset_am, how='left', on='user_id')
dataset_ff_cf_lb_am.head(5)

In [23]:
#SE LE AGREGA EL TARGET AL DATASET FINAL (si, hay que levantar todo esto)

df_pageviews = pd.read_csv("./data/pageviews.csv",
                   parse_dates=["FEC_EVENT"])
df_pageviews_complemento = pd.read_csv("./data/pageviews_complemento.csv",
                   parse_dates=["FEC_EVENT"])
df_pageviews = pd.concat([df_pageviews, df_pageviews_complemento],ignore_index=True,sort=False).drop_duplicates().reset_index(drop=True)
df_conversiones = pd.read_csv("./data/conversiones.csv").drop_duplicates().reset_index(drop=True)

In [24]:
def universo_train(df):
    universo = pd.DataFrame()
    universo['user_id'] = df.USER_ID.unique()
    universo['snapshot_mes'] = mes_snapshot
    return universo

def target(df):
    df = pd.merge(df, df_conversiones, how='left', left_on=['user_id'], right_on=['USER_ID'])
    df.mes.fillna(value= 99, inplace=True)
    df = df.astype({'mes': 'int32'}) 
    
    df['target'] = 0
    df.loc[df.mes == df.snapshot_mes +1, 'target'] = 1
    df.loc[df.mes == df.snapshot_mes +2, 'target'] = 1
    df.loc[df.mes == df.snapshot_mes +3, 'target'] = 1

    return (df.groupby(['user_id', 'snapshot_mes'])['target'].max()).to_frame().reset_index()

In [26]:
mes_snapshot = 9  #9 para train 12 para scoreo

In [33]:
universo = universo_train(df_pageviews)
target_universo = target(universo)
target_universo = target_universo.drop(['snapshot_mes'], axis = 1)

#dataset_ff_cf = pd.merge(dataset_ff_cf, target_universo, how='left', on=['user_id'])
dataset_ff_cf_lb_am_T = pd.merge(dataset_ff_cf_lb_am, target_universo, how='left', on=['user_id'])

In [38]:
#Prueba de cuantos 1 y 0 tenemos en el datasets
#dataset_ff_cf.groupby('target').count()
dataset_ff_cf_lb_am_T.groupby('target').count()

,user_id,qty_adopciones_1M,qty_adopciones_3M,qty_adopciones_6M,qty_adopciones_9M,mes,anio,qty_dias_ultima_adopcion,qty_dias_ultimo_page_ingreso,snapshot_mes,FEC_EVENT_MAX,CANT_TOTAL_EVER,CANT_TOTAL_ULT_7D,FEC_EVENT_MENOS_7D,RATIO_VISITA_MADRUGADA,RATIO_VISITA_MANANA,RATIO_VISITA_TARDE,RATIO_VISITA_NOCHE
target,,,,,,,,,,,,,,,,,,
0,11140,11140,11140,11140,11140,11140,11140,11140,11140,11140,11140,11140,11140,11140,11140,11140,11140,11140
1,389,389,389,389,389,389,389,389,389,389,389,389,389,389,389,389,389,389


In [ ]:
# EXPORTO EL DATASET FINAL
dataset_ff_cf_lb_am_T.to_csv('dataset_ff_cf_lb_am_T.csv', index = None, header=True)